In [1]:
import os, sys                               # system functions
import nipype.interfaces.io as nio           # Data i/o
from nipype.interfaces.io import DataSink
import nipype.interfaces.fsl as fsl          # fsl
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.interfaces.utility as util     # utility
import nipype.algorithms.modelgen as model   # model generation
import errno
import glob
import pandas as pd

fsl.FSLCommand.set_default_output_type('NIFTI_GZ')

# updating nipype from 1.7.0 -> 1.8.1, will hopefully ifx no PE estimates founds error # didn't fix

230203-14:26:22,945 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.7.0


### Copes at level 2:
##### Model 0:
1. FS
2. SS
3. Go
4. FS-Go
5. FS-SS
6. SS-Go

##### Model 1:
1. response_left
2. response_right
3. response_left-response_right

NOTE: had to manually change reg expression for this dataset as fwhm name was longer than all others

In [2]:
# dataset = 'Leipzig_7T_SM'
# dataset = 'Leipzig_7T_GdH'
dataset = 'aron_3T'

#### !!! must manually change subject numbers in nipype code based on dataset !!!

In [3]:
# general set-up
project_folder = '/home/scotti/projects/3t_7t_sst_comparison'

if dataset == 'Leipzig_7T_SM':
    task = 'stop'
    spaces = ['T1w'] 
    model_ns = ['0']
    fwhms = ['1p6']
    
elif dataset == 'Leipzig_7T_GdH':
    task = 'stop'
    spaces = ['T1w'] 
    model_ns = ['0']
    fwhms = ['1p5']
    
elif dataset == 'aron_3T':
    task = 'stopsignal'
    spaces = ['T1w'] 
    model_ns = ['0']
    fwhms = ['5p0']#['3p125']
    
if model_ns[0] == '0':
    contrasts = ['0','1','2','3','4','5']  # task from second level model
elif model_ns[0] == '1':
    contrasts = ['0','1','2'] # motor from second level model

subject_ids = [x.split('/')[-4].split('-')[-1] for x in sorted(glob.glob(os.path.join(project_folder, 'derivatives', 'glm_feat_sst_roi', 
                                     'subject_level_model', dataset, 'sub-*', 'func',f'model-{model_ns[0]}',
                                     f'*task-{task}_space-MNI*contrast-0_desc-zstat*')))]
work_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders', dataset)
slm_folder = os.path.join(project_folder, 'derivatives', 'glm_feat_sst_roi', 'subject_level_model', dataset)

template_brain = os.path.join(project_folder, 'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
template_brain_mask = os.path.join(project_folder, 'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

output_dir = os.path.join(project_folder, 'derivatives', "glm_feat_sst_roi", "group_level_model", dataset,f'model-{model_ns[0]}')

templates = {'level2_cope': os.path.join(slm_folder, 'sub-*', 'func', 'model-{model_n}', '*task-stopsignal_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-cope.nii.gz'),
             'level2_varcope': os.path.join(slm_folder, 'sub-*', 'func', 'model-{model_n}', '*task-stopsignal_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-varcope.nii.gz'),
             'level2_tdof': os.path.join(slm_folder, 'sub-*', 'func', 'model-{model_n}', '*task-stopsignal_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-tdof_t.nii.gz')}


In [4]:
print(f"""
dataset : {dataset}
model_n : {model_ns}
task : {task}
no. subs : {len(subject_ids)}
subs : {subject_ids}
work_dir : {work_dir}
slm folder : {slm_folder}
output dir : {output_dir}
contrasts : {contrasts}

## if uneven number of template / designs / subs the code will crash
no.copes = {len(glob.glob(templates['level2_cope'].format(model_n=model_ns[0],contrast_n=0)))}
no.varvopes = {len(glob.glob(templates['level2_varcope'].format(model_n=model_ns[0],contrast_n=0)))}
no.tdofs = {len(glob.glob(templates['level2_tdof'].format(model_n=model_ns[0],contrast_n=0)))}
""")


dataset : aron_3T
model_n : ['0']
task : stopsignal
no. subs : 14
subs : ['01', '02', '03', '04', '05', '06', '07', '09', '10', '11', '12', '13', '14', '15']
work_dir : /home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T
slm folder : /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/subject_level_model/aron_3T
output dir : /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/group_level_model/aron_3T/model-0
contrasts : ['0', '1', '2', '3', '4', '5']

## if uneven number of template / designs / subs the code will crash
no.copes = 14
no.varvopes = 14
no.tdofs = 14



In [5]:
# ## make brain mask for flameo
# import nilearn
# from nilearn import plotting
# import nibabel as nib

# hdr = nib.load('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
# brain_data = hdr.get_fdata()
# brain_data[brain_data>0] = 1
# brain_mask = nib.Nifti1Image(brain_data, affine=hdr.affine, header=hdr.header)
# brain_mask.to_filename('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

In [6]:
##
workflow = pe.Workflow(name='feat_level3_sst_roi')
workflow.base_dir = work_dir
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# Identity
identity = pe.Node(util.IdentityInterface(fields=['contrast_n', 'model_n']), name='identity')
identity.iterables = [('contrast_n', contrasts),
                      ('model_n', model_ns)]

# Selector
selector = pe.Node(nio.SelectFiles(templates), name='selector')
workflow.connect(identity, 'contrast_n', selector, 'contrast_n')
workflow.connect(identity, 'model_n', selector, 'model_n')

## Merge copes, varcopes, masks
copemerge    = pe.Node(interface=fsl.Merge(dimension='t'),
                          name="copemerge")

varcopemerge = pe.Node(interface=fsl.Merge(dimension='t'),
                       name="varcopemerge")

workflow.connect(selector, 'level2_cope', copemerge, 'in_files')
workflow.connect(selector, 'level2_varcope', varcopemerge, 'in_files')

In [7]:
def get_subject_ids():
#     if dataset == 'Leipzig_7T_SM':
    subject_ids = ['01', '02', '03', '04', '05', '06', '07', '09', '10', '11', '12', '13', '14', '15']
        
#     elif dataset == 'Leipzig_7T_GdH':
#     subject_ids = ['BI3T', 'DA9T', 'FMFT', 'GAIT', 'HCBT', 'KCAT', 'KP6T', 'LV2T', 'MRCT', 'NM3T', 'PF5T', 
#                        'RSIT', 'SC1T', 'SPGT', 'TS6T', 'UM2T', 'VL1T', 'WSFT', 'WW2T', 'ZK4T']
        
    return subject_ids

subject_id_getter = pe.Node(util.Function(output_names=['subject_ids'],
                                          function=get_subject_ids),
                            name='subject_id_getter')

def get_design_matrix(second_level_contrast, subject_ids): #=subject_ids):
    print(second_level_contrast)
    regressors = {'intercept': [1 for x in range(len(subject_ids))]}
    
    # contrasts (3rd-level)
    third_level_contrasts = [('Group mean', 'T', ['intercept'], [1.0]),
                             ('-Group mean', 'T', ['intercept'], [-1.0])
                             ]
    
    return third_level_contrasts, regressors


contrastgen_l3 = pe.Node(util.Function(input_names=['second_level_contrast', 'subject_ids'],
                                       output_names=['third_level_contrasts', 'regressors'],
                                       function=get_design_matrix),
                      name='contrastgen_l3')

level3model = pe.Node(interface=fsl.MultipleRegressDesign(),
                      name='l3model')


workflow.connect(subject_id_getter, 'subject_ids', contrastgen_l3, 'subject_ids')
workflow.connect(identity, 'contrast_n', contrastgen_l3, 'second_level_contrast')
workflow.connect(contrastgen_l3, 'third_level_contrasts', level3model, 'contrasts')
workflow.connect(contrastgen_l3, 'regressors', level3model, 'regressors')



In [8]:
# flameo = pe.Node(
#     interface=fsl.FLAMEO(run_mode='flame1'),
#     name="flame1")

# flameo.inputs.mask_file = template_brain_mask

# workflow.connect([
#     (copemerge, flameo, [('merged_file', 'cope_file')]),
#     (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
#     (level3model, flameo, [('design_mat', 'design_file'),
#                            ('design_con', 't_con_file'), 
#                            ('design_grp', 'cov_split_file')]),
# ])

flameo = pe.Node(
    interface=fsl.FLAMEO(),
    name="flameo")

flameo.iterables = ('run_mode', ['flame1', 'flame12'])
flameo.inputs.mask_file = template_brain_mask
flameo.inputs.infer_outliers = False   # run with automatic outlier detection

workflow.connect([
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level3model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), 
                           ('design_grp', 'cov_split_file')]),
])

In [9]:
## cluster thresholding
# Smoothness estimation
smoothestimate = pe.MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothestimate')
smoothestimate.inputs.mask_file = template_brain_mask

workflow.connect(flameo, 'zstats', smoothestimate, 'zstat_file')

# get volume
get_volume = pe.Node(fsl.ImageStats(op_string = '-V'), name='get_volume')
get_volume.inputs.in_file = template_brain_mask


# Cluster threshold
grf_cluster = pe.MapNode(fsl.Cluster(), iterfield=['dlh', 'in_file'], name='grf_cluster')
grf_cluster.iterables = [("threshold", [2.3, 3.1])]
grf_cluster.inputs.out_localmax_txt_file = True
grf_cluster.inputs.pthreshold = 0.05
grf_cluster.inputs.out_index_file = True
grf_cluster.inputs.out_threshold_file = True


def volume_convert(input):
    return int(input[0])

workflow.connect(get_volume, ('out_stat', volume_convert), grf_cluster, 'volume')
workflow.connect(smoothestimate, 'dlh', grf_cluster, 'dlh')
workflow.connect(flameo, 'zstats', grf_cluster, 'in_file')

In [10]:
# ## Datasink
# datasink = pe.Node(nio.DataSink(), name='sinker')
# datasink.inputs.base_directory=os.path.join(project_folder, 'derivatives', "glm_feat", "group_level_model", "ses-sstmsit","task-sst")

# workflow.connect(copemerge, 'merged_file', datasink, 'copes_merged')
# workflow.connect(level3model, 'design_con', datasink, 'design_con')
# workflow.connect(level3model, 'design_mat', datasink, 'design_mat')

# ## todo: substitutions
# workflow.connect(flameo, 'zstats', datasink, 'third_level_model.level3_zstats')
# workflow.connect(flameo, 'copes', datasink, 'third_level_model.level3_copes')
# workflow.connect(flameo, 'var_copes', datasink, 'third_level_model.level3_varcopes')
# workflow.connect(flameo, 'tdof', datasink, 'third_level_model.level3_tdof_ts')

# ## cluster results
# workflow.connect(grf_cluster, 'threshold_file', datasink, 'grf_thresholded_zstats_file')
# workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'grf_localmax_txt_file')
# workflow.connect(grf_cluster, 'index_file', datasink, 'cluster_indices')

In [11]:
## Datasink
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory=output_dir


substitutions_regexp = [(r'third_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
                         'model-\\2/model-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\7_flame-\\3_desc-\\6_voxelthreshold-\\4.nii.gz'),
                        (r'third_level_model/level3_.*/_contrast_n_(\d+)_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
                          'model-\\2/model-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\5_flame-\\3_desc-\\4.nii.gz'),
                        # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
                        #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
                       ]

datasink.inputs.regexp_substitutions = substitutions_regexp

## todo: substitutions
workflow.connect(flameo, 'zstats', datasink, 'third_level_model.level3_zstats')
workflow.connect(flameo, 'copes', datasink, 'third_level_model.level3_copes')
workflow.connect(flameo, 'var_copes', datasink, 'third_level_model.level3_varcopes')
workflow.connect(flameo, 'tdof', datasink, 'third_level_model.level3_tdof_ts')

## cluster results
workflow.connect(grf_cluster, 'threshold_file', datasink, 'third_level_model.grf_thresholded_zstats_file')
workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'third_level_model.grf_localmax_txt_file')
workflow.connect(grf_cluster, 'index_file', datasink, 'third_level_model.grf_cluster_indices')

In [12]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs': 24, 'memory_gb': 240})

230203-14:26:25,443 nipype.workflow INFO:
	 Workflow feat_level3_sst_roi settings: ['check', 'execution', 'logging', 'monitoring']
230203-14:26:25,494 nipype.workflow INFO:
	 Running in parallel.
230203-14:26:25,562 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 8 jobs ready. Free memory (GB): 240.00/240.00, Free processors: 24/24.
230203-14:26:25,677 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_roi.subject_id_getter" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_roi/subject_id_getter".
230203-14:26:25,678 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_roi.get_volume" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_roi/get_volume".
230203-14:26:25,679 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_roi.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_roi/_co

230203-14:26:27,634 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_roi.varcopemerge" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_roi/_contrast_n_2_model_n_0/varcopemerge".
230203-14:26:27,635 nipype.workflow INFO:
	 [Node] Executing "contrastgen_l3" <nipype.interfaces.utility.wrappers.Function>
230203-14:26:27,634 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_roi.copemerge" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_roi/_contrast_n_4_model_n_0/copemerge".
230203-14:26:27,634 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_roi.varcopemerge" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_roi/_contrast_n_4_model_n_0/varcopemerge".
230203-14:26:27,635 nipype.workflow INFO:
	 [Node] Executing "contrastgen_l3" <nipype.interfaces.utility.wrappers.Function>
230203-14:26:27,634 

230203-14:26:31,572 nipype.workflow INFO:
	 [Job 30] Completed (feat_level3_sst_roi.l3model).
230203-14:26:31,573 nipype.workflow INFO:
	 [Job 31] Completed (feat_level3_sst_roi.l3model).
230203-14:26:31,574 nipype.workflow INFO:
	 [MultiProc] Running 12 tasks, and 0 jobs ready. Free memory (GB): 237.60/240.00, Free processors: 12/24.
                     Currently running:
                       * feat_level3_sst_roi.varcopemerge
                       * feat_level3_sst_roi.copemerge
                       * feat_level3_sst_roi.varcopemerge
                       * feat_level3_sst_roi.copemerge
                       * feat_level3_sst_roi.varcopemerge
                       * feat_level3_sst_roi.copemerge
                       * feat_level3_sst_roi.varcopemerge
                       * feat_level3_sst_roi.copemerge
                       * feat_level3_sst_roi.varcopemerge
                       * feat_level3_sst_roi.copemerge
                       * feat_level3_sst_roi.varcopemerge


230203-14:26:53,453 nipype.workflow INFO:
	 [Node] Finished "flameo", elapsed time 19.809682s.
230203-14:26:53,490 nipype.workflow INFO:
	 [Node] Finished "flameo", elapsed time 19.838959s.
230203-14:26:53,589 nipype.workflow INFO:
	 [Job 36] Completed (feat_level3_sst_roi.flameo).
230203-14:26:53,590 nipype.workflow INFO:
	 [Job 42] Completed (feat_level3_sst_roi.flameo).
230203-14:26:53,592 nipype.workflow INFO:
	 [MultiProc] Running 10 tasks, and 2 jobs ready. Free memory (GB): 238.00/240.00, Free processors: 14/24.
                     Currently running:
                       * feat_level3_sst_roi.flameo
                       * feat_level3_sst_roi.flameo
                       * feat_level3_sst_roi.flameo
                       * feat_level3_sst_roi.flameo
                       * feat_level3_sst_roi.flameo
                       * feat_level3_sst_roi.flameo
                       * feat_level3_sst_roi.flameo
                       * feat_level3_sst_roi.flameo
                   

230203-14:26:57,649 nipype.workflow INFO:
	 [Node] Setting-up "_smoothestimate0" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_roi/_contrast_n_4_model_n_0/_run_mode_flame1/smoothestimate/mapflow/_smoothestimate0".
230203-14:26:57,650 nipype.workflow INFO:
	 [Node] Executing "_smoothestimate1" <nipype.interfaces.fsl.model.SmoothEstimate>
230203-14:26:57,648 nipype.workflow INFO:
	 [Node] "_smoothestimate0" found cached.
230203-14:26:57,648 nipype.workflow INFO:
	 [Node] Setting-up "_smoothestimate1" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_roi/_contrast_n_2_model_n_0/_run_mode_flame1/smoothestimate/mapflow/_smoothestimate1".
230203-14:26:57,650 nipype.workflow INFO:
	 [Node] Setting-up "_smoothestimate1" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_roi/_contrast_n_4_model_n_0/_run_mode_flame1/smoothestim

...

230203-14:42:42,584 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/group_level_model/aron_3T/model-0/third_level_model/level3_zstats/_contrast_n_0_model_n_0/_run_mode_flame12/zstat1.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/group_level_model/aron_3T/model-0/model-0/model-0_subjectlevelcontrast-0_grouplevelcontrast-1_flame-12_desc-zstat.nii.gz
230203-14:42:42,585 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/group_level_model/aron_3T/model-0/third_level_model/level3_copes/_contrast_n_0_model_n_0/_run_mode_flame12/cope1.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/group_level_model/aron_3T/model-0/model-0/model-0_subjectlevelcontrast-0_grouplevelcontrast-1_flame-12_desc-cope.nii.gz
230203-14:42:42,585 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/g

230203-14:42:42,591 nipype.workflow INFO:
	 [Node] Finished "sinker", elapsed time 0.008566s.
230203-14:42:42,592 nipype.workflow INFO:
	 [Node] Finished "sinker", elapsed time 0.008905s.
230203-14:42:43,364 nipype.workflow INFO:
	 [Node] Finished "_grf_cluster1", elapsed time 0.781389s.
230203-14:42:43,404 nipype.workflow INFO:
	 [Node] Finished "_grf_cluster0", elapsed time 0.825561s.
230203-14:42:43,426 nipype.workflow INFO:
	 [Node] Finished "_grf_cluster1", elapsed time 0.847621s.
230203-14:42:43,473 nipype.workflow INFO:
	 [Node] Finished "_grf_cluster0", elapsed time 0.895572s.
230203-14:42:44,517 nipype.workflow INFO:
	 [Job 82] Completed (feat_level3_sst_roi.sinker).
230203-14:42:44,518 nipype.workflow INFO:
	 [Job 83] Completed (feat_level3_sst_roi.sinker).
230203-14:42:44,519 nipype.workflow INFO:
	 [Job 168] Completed (_grf_cluster0).
230203-14:42:44,520 nipype.workflow INFO:
	 [Job 169] Completed (_grf_cluster1).
230203-14:42:44,520 nipype.workflow INFO:
	 [Job 170] Comple

230203-14:42:46,580 nipype.workflow INFO:
	 [Node] Cached "_grf_cluster1" - collecting precomputed outputs
230203-14:42:46,580 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/group_level_model/aron_3T/model-0/third_level_model/level3_zstats/_contrast_n_1_model_n_0/_run_mode_flame12/zstat1.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/group_level_model/aron_3T/model-0/model-0/model-0_subjectlevelcontrast-1_grouplevelcontrast-1_flame-12_desc-zstat.nii.gz
230203-14:42:46,580 nipype.workflow INFO:
	 [Node] "_grf_cluster1" found cached.
230203-14:42:46,580 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/group_level_model/aron_3T/model-0/third_level_model/level3_zstats/_contrast_n_1_model_n_0/_run_mode_flame12/zstat2.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/group_level_model/aron_3T/model-0/model-0/model-0_sub

230203-14:42:46,586 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/group_level_model/aron_3T/model-0/third_level_model/grf_thresholded_zstats_file/_contrast_n_1_model_n_0/_run_mode_flame12/_threshold_3.1/_grf_cluster1/zstat2_threshold.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/group_level_model/aron_3T/model-0/model-0/model-0_subjectlevelcontrast-1_grouplevelcontrast-2_flame-12_desc-zstat_voxelthreshold-3.1.nii.gz
230203-14:42:46,587 nipype.workflow INFO:
	 [Node] Finished "sinker", elapsed time 0.00716s.
230203-14:42:46,588 nipype.workflow INFO:
	 [Node] Finished "sinker", elapsed time 0.007437s.
230203-14:42:48,521 nipype.workflow INFO:
	 [Job 66] Completed (feat_level3_sst_roi.grf_cluster).
230203-14:42:48,523 nipype.workflow INFO:
	 [Job 67] Completed (feat_level3_sst_roi.grf_cluster).
230203-14:42:48,524 nipype.workflow INFO:
	 [Job 86] Completed (feat_level3_sst_roi.sinker).
230203-14:42

230203-14:42:48,581 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/group_level_model/aron_3T/model-0/third_level_model/grf_thresholded_zstats_file/_contrast_n_2_model_n_0/_run_mode_flame12/_threshold_2.3/_grf_cluster0/zstat1_threshold.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/group_level_model/aron_3T/model-0/model-0/model-0_subjectlevelcontrast-2_grouplevelcontrast-1_flame-12_desc-zstat_voxelthreshold-2.3.nii.gz
230203-14:42:48,581 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/group_level_model/aron_3T/model-0/third_level_model/level3_tdof_ts/_contrast_n_2_model_n_0/_run_mode_flame12/tdof_t2.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_sst_roi/group_level_model/aron_3T/model-0/model-0/model-0_subjectlevelcontrast-2_grouplevelcontrast-2_flame-12_desc-tdof_t.nii.gz
230203-14:42:48,581 nipype.interface INFO:
	 

In [15]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_roi/_contrast_n_0_model_n_0/_run_mode_flame1/_threshold_3.1/sinker/result_sinker.pklz')
res

In [19]:
res.outputs